<a href="https://colab.research.google.com/github/pizzapuppies/Usif_FYP_code/blob/main/RD_hyper_tuned_concentration_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q -U keras-tuner

In [ ]:
from google.colab import drive
import pandas as pd 
import os
import tensorflow as tf
import numpy as np  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import csv
import random
from tensorflow import keras
import sys
import keras_tuner as kt


#Data preprocessing

The data in the csv needs to transposed and split into training and testing

In [ ]:
#@title spliting the data
train_test_split = "10%" #@param ["10%", "20%"]
data = []
with open("/content/FYP Dengue data - Sheet1(3).csv","r") as rfile:
  line = 0

  reader  = csv.reader(rfile)
  real,imag = [],[]
  
  for i in reader:

    if line> 55:
      #print(line,i)
      if line%3 == 0:

        imag = i[3:53]
        label = f"p{i[1]}" if i[2] == "1" else f"n{i[1]}"
        data.append([label]+ real+imag)
        real,imag = [],[]
      elif line%3 == 2:
        real = i[3:53]
        

    line += 1

data = np.array(data)

test,train = [],[]
count = random.randint(0,10)

spl = 10 if train_test_split == '10%' else 5

for i in data:
  if count%spl == 0:  ## This line controls the train_test split (10 for 10%, 5 for 20%)
    test.append(i)
  else: train.append(i) 
  count += 1

test = np.array(test)
train = np.array(train)

with open("train_data.csv","w") as wfile:
  writer = csv.writer(wfile)
  for i in train:
    writer.writerow(i)
with open("test_data.csv","w") as wfile:
  writer = csv.writer(wfile)
  for i in test:
    writer.writerow(i)

In [ ]:
#@title extracting the training data + one hot encoding the labels
y = []
x = []

with open("train_data.csv","r") as tfile:
  reader = csv.reader(tfile)
  for i in reader:
    
    y.append(i[0])

    a,b = list(map(float,i[1:51])),list(map(float,i[51:]))
    x.append([a,b])

x = np.array(x)
y = np.array(y)


temp = list(zip(x, y))
random.shuffle(temp)
x, y = zip(*temp)
x, y = list(x), list(y)
x = np.array(x)
y = np.array(y)

print("the shape of X is",x.shape)


#@title one hot encoding Y

mapping = {}
codes = list(set(y))
l = len(codes)

for i in range(l):
  li = [0]*l
  
  li[i] = 1
  
  mapping[codes[i]] = np.array(li)

temp = []
for i in y:
  temp.append(mapping[i])

y = np.array(temp)

print("The shape of Y is",y.shape)


the shape of X is (126, 2, 50)
The shape of Y is (126, 14)


# building and runnning the model

In [ ]:
def model_builder(hp):
  model = tf.keras.models.Sequential()
  model.add(keras.layers.Dense(128,         #Number of nodes
                        input_shape=(2,50), #Number of input variables
                        name='Hidden-Layer-1', #Logical name
                        activation='relu'))    #activation function

  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  
  #Add a second hidden layer
  model.add(keras.layers.Dense(units=hp_units,
                                name='Hidden-Layer-2',
                                activation='relu',
                                ))

  

  #Add a second hidden layer
  model.add(keras.layers.Dense(128,
                                name='Hidden-Layer-3',
                                activation='relu'))



  #Add a second hidden layer
  model.add(keras.layers.Dense(128,
                                name='Hidden-Layer-4',
                              activation='relu'))
  
  model.add(keras.layers.Dense(128,
                                name='Hidden-Layer-5',
                              activation='relu'))
  
  model.add(keras.layers.Dense(128,
                                name='Hidden-Layer-6',
                              activation='relu'))

  model.add(tf.keras.layers.Flatten())
  model.add(keras.layers.Dense(14,
                             name='Output-Layer',
                             activation='softmax'))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  
  

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

  return model

def learn_test(X_train,Y_train,X_test,Y_test):
  

  tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

  tuner.search(X_train, Y_train, epochs=150, validation_split=0.2, callbacks=[stop_early])

  # Get the optimal hyperparameters
  best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

  #print(f
  """
  The hyperparameter search is complete. The optimal number of units in the first densely-connected
  layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
  is {best_hps.get('learning_rate')}.
  )"""

 # Build the model with the optimal hyperparameters and train it on the data for 50 epochs
  model = tuner.hypermodel.build(best_hps)
  history = model.fit(X_train, Y_train, epochs=50, validation_split=0.2)

  val_acc_per_epoch = history.history['val_accuracy']
  best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
  print('Best epoch: %d' % (best_epoch,))

  hypermodel = tuner.hypermodel.build(best_hps)

  # Retrain the model
  hypermodel.fit(X_train, Y_train, epochs=best_epoch, validation_split=0.2)

  eval_result = hypermodel.evaluate(X_test, Y_test)
  print("[test loss, test accuracy]:", eval_result)
  return eval_result[1]


In [ ]:
y_test = []
x_test = []
with open("test_data.csv","r") as tfile:
  reader = csv.reader(tfile)
  for i in reader:
    
    y_test.append(i[0])
    x_test.append([i[1:51],i[51:]])

x_test = np.array(x)
y_test = np.array(y)


temp = list(zip(x, y))
random.shuffle(temp)
x_test, y_test = zip(*temp)
x_test, y_test = list(x), list(y)
x_test = np.array(x)
y_test = np.array(y)

In [ ]:
xa = learn_test(x,y,x_test,y_test)
print("accuracy is ",round(xa*100))

Trial 30 Complete [00h 00m 03s]
val_accuracy: 0.19230769574642181

Best val_accuracy So Far: 0.26923078298568726
Total elapsed time: 00h 01m 53s
Epoch 1/50
4/4 [==============================] - 2s 134ms/step - loss: 483.7935 - accuracy: 0.0700 - val_loss: 241.5589 - val_accuracy: 0.1538
Epoch 2/50
4/4 [==============================] - 0s 18ms/step - loss: 246.4109 - accuracy: 0.1100 - val_loss: 118.7761 - val_accuracy: 0.1154
Epoch 3/50
4/4 [==============================] - 0s 19ms/step - loss: 145.7537 - accuracy: 0.1300 - val_loss: 89.2318 - val_accuracy: 0.1154
Epoch 4/50
4/4 [==============================] - 0s 16ms/step - loss: 109.2523 - accuracy: 0.1100 - val_loss: 62.0855 - val_accuracy: 0.2308
Epoch 5/50
4/4 [==============================] - 0s 18ms/step - loss: 80.6291 - accuracy: 0.0600 - val_loss: 46.8305 - val_accuracy: 0.0000e+00
Epoch 6/50
4/4 [==============================] - 0s 16ms/step - loss: 61.5405 - accuracy: 0.0800 - val_loss: 39.2910 - val_accuracy: 0.076